In [1]:

%matplotlib inline

from pathlib import Path
from datetime import date
import json
from io import BytesIO
from zipfile import ZipFile, BadZipFile
from tqdm import tqdm
import requests
import numpy as np
import pandas as pd

from pprint import pprint

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import os
import multiprocessing

In [70]:
sns.set_style('whitegrid')
# store data in this directory since we won't use it in other chapters
data_path = Path(os.getcwd()) # perhaps set to external harddrive to accomodate large amount of data

def filter_by_std(df, num_std):
    if df.shape[0] == 1:
        return df
    else:
        return df[np.abs(df-df.mean()) <= num_std*df.std()]

def mean_calc(df, columns, identifier, time):
    df = df[columns+identifier]
    grouped = df.groupby(identifier)
    filtered_df = grouped.apply(lambda group: filter_by_std(group, num_std=1.5))
    group_means = filtered_df.groupby(identifier).mean()
    group_means["Time"] = time
    return group_means

data_path

WindowsPath('c:/Users/fuway/Desktop/stocks')

In [71]:
#debt
'debtToAssets', 'quickRatio', 'CurrentRatio', 'DebtToEquity',
#efficiency
'AssetsTurnover','pbRatio', 'priceToFreeCashFlowsRatio','priceToSalesRatio','ReturnOnAssets', 'ReturnOnEquity','peRatio', 'roic',
#margin
'OperatingMargin', 'NetProfitMargin','grossProfitMargin', 
#growth
'epsdilutedGrowth','OperatingMarginIncrease','NetProfitMarginIncrease','NetIncomeGrowth','RevenueGrowth', 'OperatingIncomeGrowth', 'GrossProfitGrowth'

('epsdilutedGrowth',
 'OperatingMarginIncrease',
 'NetProfitMarginIncrease',
 'NetIncomeGrowth',
 'RevenueGrowth',
 'OperatingIncomeGrowth',
 'GrossProfitGrowth')

In [78]:
fund_path = data_path/"fundamental data"/"year ratios"
ratio_columns = ['debtToAssets', 'quickRatio', 'CurrentRatio', 'DebtToEquity','AssetsTurnover','pbRatio', 'priceToFreeCashFlowsRatio','priceToSalesRatio','ReturnOnAssets', 'ReturnOnEquity','peRatio', 'roic','OperatingMargin', 'NetProfitMargin','grossProfitMargin', 'epsdilutedGrowth','OperatingMarginIncrease','NetProfitMarginIncrease','NetIncomeGrowth','RevenueGrowth', 'OperatingIncomeGrowth', 'GrossProfitGrowth']
identifiers = [["Sector"],["Sector","Industry"]]
for year in range(2009,2024):
# for year in [2009]:
    str_year = str(year)
    str_file_year = str_year+"_ratios_Year.csv"
    file_path = fund_path/str_year/str_file_year
    year_data = pd.read_csv(file_path)
    for i in identifiers:
        data = mean_calc(year_data, ratio_columns,i,year)
        new_file = str_year+"_ratios_Year_"+"_".join(i)+".csv"
        file_path = fund_path/str_year/new_file
        data.to_csv(file_path)
    for q in ["Q1", "Q2", "Q3", "Q4"]:
        str_file_Q = str_year+q+"_ratios_Q.csv"
        file_path = fund_path/str_year/str_file_Q
        if file_path.exists():
            Q_data = pd.read_csv(file_path)
            for i in identifiers:
                data = mean_calc(Q_data, ratio_columns,i,str_year+q)
                new_file = str_year+q+"_ratios_Q_"+"_".join(i)+".csv"
                file_path = fund_path/str_year/new_file
                data.to_csv(file_path)
data

debtToAssets  quickRatio  \
Sector          Industry                                                 
Basic Materials Building Materials                0.324181    0.872423   
                Gold                              0.000000    8.213232   
                Silver                            0.000861    4.990276   
                Specialty Chemicals               0.103709    5.240585   
                Steel                             0.016602   12.763871   
...                                                    ...         ...   
Technology      Semiconductors                    0.162327    2.193592   
                Software—Application              0.127245    1.576476   
                Software—Infrastructure           0.222838    1.175178   
Utilities       Utilities—Regulated Electric      0.463715    0.303176   
                Utilities—Renewable               0.690309    0.881972   

                                              CurrentRatio  DebtToEquity  \
Sector          Industry                                                   
Basic Materials Building Materials                1.995726      1.562548   
                Gold                              8.556716      0.036102   
                Silver                            5.311542      0.198277   
                Specialty Chemicals               6.513530      0.347521   
                Steel                            14.740675      0.546759   
...                                                    ...           ...   
Technology      Semiconductors                    3.236843      0.524515   
                Software—Application              1.783186      1.280579   
                Software—Infrastructure           1.339139      3.280022   
Utilities       Utilities—Regulated Electric      1.003943      2.137416   
                Utilities—Renewable               0.964763      5.883569   

                                              AssetsTurnover   pbRatio  \
Sector          Industry                                                 
Basic Materials Building Materials                  0.186978  2.210736   
                Gold                                0.000000  2.722409   
                Silver                              0.050454  1.381478   
                Specialty Chemicals                 0.244003  2.518389   
                Steel                               0.245383  9.072811   
...                                                      ...       ...   
Technology      Semiconductors                      0.120167  4.330979   
                Software—Application                0.161372  6.782192   
                Software—Infrastructure             0.116537  1.835158   
Utilities       Utilities—Regulated Electric        0.000000  0.000000   
                Utilities—Renewable                 0.031170  1.628064   

                                              priceToFreeCashFlowsRatio  \
Sector          Industry                                                  
Basic Materials Building Materials                           211.000234   
                Gold                                         -31.308918   
                Silver                                      -167.148983   
                Specialty Chemicals                          -11.199343   
                Steel                                       -156.645685   
...                                                                 ...   
Technology      Semiconductors                               120.826461   
                Software—Application                          49.854101   
                Software—Infrastructure                     -180.718151   
Utilities       Utilities—Regulated Electric                   0.000000   
                Utilities—Renewable                          -21.877053   

                                              priceToSalesRatio  \
Sector          Industry                                          
Basic Materials Build